In [13]:
import random
import math

In [14]:
#  eN - population estimate
#  M - number of marked samples before re-capture
#  C - recapture sample size
#  R - number of marked samples found in re-capture
#  eN = M*C/R , to avoid R = 0 case --> eN = ()(M+1)*(C+1)/(R+1))-1

In [15]:
myPopulationSize = 47500 

In [16]:
if (1):
    #lets define total number of rounds to simulate
    nRounds = 20
    #lets define our capture size for each recapture round
    C=[None]*nRounds
    for i in range(0,nRounds) :
        C[i]= 40
else:
# Option B - Directly specified [set above if to false to enter here]
    C=[64,32,16,8]
    nRounds = len(C)

In [23]:
# empty set to start with
setN = set()
for i in range(1,myPopulationSize):
    setN.add(i)

In [28]:
setC=[None]*nRounds
setM=[None]*nRounds
M=[None]*nRounds
R=[None]*nRounds
p=[None]*nRounds

In [27]:

#LP_eN=[None]*nRounds
#SeLP_eN=[None]*nRounds
eN=[None]*nRounds
stdev_eN=[None]*nRounds
confLow_eN=[None]*nRounds
confHigh_eN=[None]*nRounds
boundLow_eN=[None]*nRounds
boundHigh_eN=[None]*nRounds
rN=[]

print("Round\tMarked \t p \t\t est \t pest \tstdev \t\tLow \tHigh")
#Lets get our first capture
setC[0]=set(random.sample(setN,C[0]))
setM[0]=setC[0] #we mark all in round 1
rN.clear()
for r in range(1,len(C)):
    M[r] = len(setM[r-1]) #number of marked at the begining of round r
    setC[r] = set(random.sample(setN,C[r])) # Generating the Recapture round r
    R[r] = len(set(setM[r-1]&setC[r])) # number of recaptures round r
    setM[r] = set(setM[r-1]|setC[r]) # we mark the new finds in capture round r
    p[r] = round(1.0*R[r]/M[r],9) #capture probability of round r
    #Lets calculate our population estimate
    rN.append(r)
    topSum = botSum = 0

    #for i in rN :
    #  Lincoln-Petersen Estimator
    #    topSum = topSum + (M[i])*(C[i])
    #    botSum = botSum + (R[i]+1E-6)#1E-6 for zero case
    #LP_eN[r] = round(1.0*topSum/botSum) #population estimation on round r
    # 20/02/19
    #for i in rN :
    # Seber 1982 evolution of Lincoln-Petersen method (also avoids R[i] = 0 case)
    #    topSum = topSum + (M[i]+1)*(C[i]+1)
    #    botSum = botSum + (R[i]+1)
    #SeLP_eN[r] = round(1.0*topSum/botSum-len(rN)) #population estimation on round r
    # 20/02/20 : since we are going to skip R[r] = 0 cases -> back to initial variable calculation
    if p[r] > 0 :
        for i in rN :
        #  Lincoln-Petersen Estimator
            topSum = topSum + (M[i])*(C[i])
            botSum = botSum + (R[i])
        eN[r] = round(1.0*topSum/botSum) #population estimation on round r
    #eN[r] = round(1.0*topSum/botSum-len(rN)) #population estimation on round r
    #var_eN[r] = abs(round((M[r]*C[r]*(M[r]-R[r])*(C[r]-R[r]))/(R[r]*R[r]*R[r])))
    #20/02/19: added the +1 below to avoid zero case error, mostly insignificant in comparison to R[r]^3
    #var_eN[r] = abs(round((M[r]*C[r]*(M[r]-R[r])*(C[r]-R[r]))/(R[r]*R[r]*R[r]+1)))
    #20/02/20: variance estimation "Error" from original approximation is 50% undershoot if R=1;12%,R=2;4%,R=3
    # Modifying to overshoot, by how much depends on how many R[r] s' you add to both top bottom and +1 or +0.1 ,...
    # Play with relative significance of "R"'s to zero eliminating constant "+1,+0.1,..."
    # we want the zero case eliminator (+0.1) to influence as little as possible
    # {...}*(R[r]+0.01)/((R[r]^3)*R[r]+0.01) -> 0%,R=1; +0.4%,R=2; +0.3%,R=3; +0.2%,R=4
    # var_eN[r] = abs(round((M[r]*C[r]*(M[r]-R[r])*(C[r]-R[r])*(R[r]+0.01))/((R[r]*R[r]*R[r])*R[r]+0.01)))
    # 20/02/20 : since we are going to skip R[r] = 0 cases -> back to initial variable calculation
        # simplified math.sqrt(var_eN[r]) -> stdev_eN[]
        stdev_eN[r] = round(math.sqrt(abs((M[r]*C[r]*(M[r]-R[r])*(C[r]-R[r]))/(R[r]*R[r]*R[r]))))
    #Confidence Level	0.70 0.75 0.80 0.85 0.90  0.92 0.95 0.96 0.98 0.99
    #z                  1.04 1.15 1.28 1.44 1.645 0.75 1.96 2.05 2.33 2.58
        confLow_eN[r] = 1.96*math.sqrt(((1-R[r]/M[r])*(R[r]/C[r])*(1-R[r]/C[r]))/(C[r]-1))+1/(2*C[r])
        confHigh_eN[r] = R[r]/C[r]-confLow_eN[r]
        confLow_eN[r] = R[r]/C[r]+confLow_eN[r]
        boundLow_eN[r] = round(M[r]/confLow_eN[r])
        boundHigh_eN[r] = round(M[r]/confHigh_eN[r])
    #std deviation =sqrt(Var)
        #print("  %d\t %d \t %1.3f \t %d \t %d \t%3.2f \t\t%3.2f\t%3.2f"%(r,M[r],p[r],eN[r],round(C[r]/(p[r]+1E-6)),stdev_eN[r],M[r]/confLow_eN[r],M[r]/confHigh_eN[r]))
        #lets define when an estimate is valid... conditions:
        # 2*stddev -> 95% of cases contemplated
        # z= 1.96 --> 0.95 confidence level for boundries
        # 1. is estimate > 2*stddev ?
        # 2. is estimate between boundries ?
        # ?3. is pest < est ? <--- maybe --> in large numbers always falls behind
        #if (eN[r]>(stdev_eN[r])) & (eN[r]>boundLow_eN[r]) & (eN[r]<boundHigh_eN[r]):
        #boundries for large numbers doesnt seem to work out
        if (eN[r]>(stdev_eN[r])) :
            print("  %d\t %d \t %1.4f \t %d \t %d \t%d \t\t%d\t%d"%(r,M[r],p[r],eN[r],round(C[r]/(p[r])),stdev_eN[r],boundLow_eN[r],boundHigh_eN[r]))

#eN.remove(None) #None is not accepted in the function median
#print("median of last %d rounds: %d"%(int(nRounds/2),median(eN[int(nRounds/-2):])))
print("real Popualtion %d"%len(setN))

Round	Marked 	 p 		 est 	 pest 	stdev 		Low 	High
  11	 440 	 0.0023 	 105600 	 17600 	17359 		5090	-12073
  15	 599 	 0.0017 	 95920 	 23960 	23639 		6928	-16429
real Popualtion 47499
